In [2]:
import numpy as np
import laser_hockey_env as lh
import gym
from importlib import reload

In [3]:
np.set_printoptions(suppress=True)

# Normal Game Play

In [7]:
reload(lh)

<module 'laser_hockey_env' from '/home/georg/src/python/laser-hockey-rl/laserhockeyenv/laser_hockey_env.py'>

In [8]:
env = lh.LaserHockeyEnv()

have a look at the initialization condition

In [9]:
obs = env.reset()
obs_agent2 = env.obs_agent_two()
env.render()

True

In [10]:
#obs = env.reset()
obs_agent2 = env.obs_agent_two()

for _ in range(600):
    env.render()
    a1 = [1,-.5,0] # np.random.uniform(-1,1,3)
    a2 = [1,0.,0] # np.random.uniform(-1,1,3)*0    
    obs, r, d, info = env.step(np.hstack([a1,a2]))    
    obs_agent2 = env.obs_agent_two()
    if d: break

13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939

13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939894362
13.335406939

Without rendering, it runs much faster

In [20]:
obs = env.reset()
obs_agent2 = env.obs_agent_two()

for _ in range(600):    
    a1 = [1,-.5,0] # np.random.uniform(-1,1,3)
    a2 = [1,0.,0] # np.random.uniform(-1,1,3)*0    
    obs, r, d, info = env.step(np.hstack([a1,a2]))    
    obs_agent2 = env.obs_agent_two()
    if d: break

# Train Shooting

In [15]:
reload(lh)

<module 'laser_hockey_env' from '/home/georg/src/python/laser-hockey-rl/laserhockeyenv/laser_hockey_env.py'>

In [16]:
env = lh.LaserHockeyEnv(mode=lh.LaserHockeyEnv.TRAIN_SHOOTING)

In [17]:
o = env.reset()

In [18]:
env.render()

True

In [ ]:
for _ in range(60):
    env.render()
    a1 = [1,0,0] # np.random.uniform(-1,1,3)
    a2 = [0,0.,0] 
    obs, r, d, info = env.step(np.hstack([a1,a2]))    
    obs_agent2 = env.obs_agent_two()
    if d: break

# Train DEFENDING

In [4]:
reload(lh)

<module 'laser_hockey_env' from '/home/georg/src/python/laser-hockey-rl/laserhockeyenv/laser_hockey_env.py'>

In [5]:
env = lh.LaserHockeyEnv(mode=lh.LaserHockeyEnv.TRAIN_DEFENCE)

In [6]:
o = env.reset()

In [7]:
env.render()

True

In [8]:
for _ in range(60):
    env.render()
    a1 = [1,0,0] # np.random.uniform(-1,1,3)
    a2 = [0,0.,0] 
    obs, r, d, info = env.step(np.hstack([a1,a2]))    
    obs_agent2 = env.obs_agent_two()
    if d: break

Player 2 scored


# Using discrete actions

In [ ]:
reload(lh)

In [ ]:
env = lh.LaserHockeyEnv(mode=lh.LaserHockeyEnv.TRAIN_SHOOTING)

In [ ]:
import random

In [ ]:
for _ in range(200):
    env.render()
    a1_discrete = random.randint(0,7)
    a1 = env.discrete_to_continous_action(a1_discrete)
    a2 = [0,0.,0] 
    obs, r, d, info = env.step(np.hstack([a1,a2]))    
    obs_agent2 = env.obs_agent_two()
    if d: break

# Hand-crafted Opponent

In [1]:
reload(lh)

NameError: name 'reload' is not defined

In [ ]:
env = lh.LaserHockeyEnv(mode=lh.LaserHockeyEnv.TRAIN_DEFENCE)

In [ ]:
o = env.reset()

In [30]:
env.render()

True

In [37]:
player1 = lh.BasicOpponent()
player2 = lh.BasicOpponent()

In [38]:
obs_buffer = []

In [44]:
obs = env.reset()
obs_agent2 = env.obs_agent_two()
for _ in range(600):
    env.render()
    a1 = player1.act(obs)
    a2 = player2.act(obs_agent2)
    obs, r, d, info = env.step(np.hstack([a1,a2]))    
    obs_buffer.append(obs)
    obs_agent2 = env.obs_agent_two()
    if d: break

Player 2 scored


In [45]:
obs_buffer = np.asarray(obs_buffer)

In [46]:
np.mean(obs_buffer,axis=0)

array([-6.21772583,  0.14022074, -0.50388751,  0.06938814,  0.42967239,
       -0.05056231,  5.99394432,  0.190191  , -0.04062059,  1.15586982,
       -0.13772885, -0.0704963 , -0.85041827,  0.12573967, -2.2194066 ,
       -0.30853598])

In [47]:
np.std(obs_buffer,axis=0)

array([ 1.09465005,  1.7363143 ,  2.48617842,  4.01910657,  5.39319311,
        2.70587981,  1.44744218,  1.54582286,  0.24750636,  4.41002617,
        5.06345003,  0.4849688 ,  2.66636978,  2.25679011, 13.08127676,
        7.6629061 ])

In [ ]:
scaling = [ 1.0,  1.0 , 3.14, 4.0, 4.0, 2.0,  
            1.0,  1.0,  3.14, 4.0, 4.0, 2.0,  
            2.0, 2.0, 10.0, 10.0]

# Human Opponent

In [10]:
reload(lh)

<module 'laser_hockey_env' from '/home/georg/src/python/laser-hockey-rl/laserhockeyenv/laser_hockey_env.py'>

In [11]:
env = lh.LaserHockeyEnv()

In [12]:
o = env.reset()

In [13]:
env.render()

True

In [14]:
player1 = lh.HumanOpponent(env=env, player=1)
player2 = lh.BasicOpponent()

Human Controls:
 left:			left arrow key left
 right:			arrow key right
 up:			arrow key up
 down:			arrow key down
 tilt clockwise:	w
 tilt anti-clockwise:	s


In [18]:
obs = env.reset()
obs_agent2 = env.obs_agent_two()
for _ in range(600):
    env.render()
    a1 = player1.act(obs)
    a2 = player2.act(obs_agent2)
    obs, r, d, info = env.step(np.hstack([a1,a2]))    
    obs_agent2 = env.obs_agent_two()
    if d: break

Player 1 scored
